In [23]:
%reset
%matplotlib inline
import parcels
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4_3D, AdvectionRK4, ParticleFile, ErrorCode, Variable, VectorField, Field, FieldSamplingError, plotTrajectoriesFile, GeographicPolar, Geographic, logger
from parcels import TimeConverter, ScipyParticle
import numpy as np
import xarray as xr
import math
from math import sqrt
import matplotlib
from matplotlib import pyplot as plt, animation
import datetime as dati
import time as tm
import pandas as pd
import os

def GeneralStats(array):
    stats = np.nanmean(array), np.nanstd(array), np.nanmin(array), np.nanmax(array)
    return stats

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Files and paths

In [24]:
path = "E:\\OceanParcels" # for working on home computer

month = np.array(["may", "jun", "jul", "aug", "sep", "oct"])
coordfile = path + "\\data\\processed\\coordinates.nc"

gr_raw = path + "\\data\\raw\\reindexed_" # reindexed raw grid data
gr_int = path + "\\data\\processed\\interim grids\\reindexed_"
gr_fin = path + "\\data\\processed\\final grids\\final_" # final grids
sm_path = path + "\\data\\processed\\summary stats\\"
sc_path = path + "\\data\\processed\\scenario data\\"
densitydata = path + "\\data\\raw\\year_D2.csv"

scenario = np.array(["S0", "S1", "S2", "S3", "S4", "S5", "S6"])
days = np.array([31, 30, 31, 31, 30, 31])
cday = np.array([1, 32, 62, 93, 124, 154])

canopy_range = range(4,10)
under_range = range(11,21)

# Average velocity

Calculate and save average velocity (unattenuated) for each month. This outputs a 3D (depth-averagd) array of lat, lon, and time.

In [25]:
%%script echo Skipped!

for mo in range(0,6):
    print("Starting " + month[mo] + "...")
    
    # Open file
    ds = xr.open_dataset(gr_raw + month[mo] + ".nc").load()    
    # Simple variables
    Uav = ds.Uav.values
    Vav = ds.Vav.values
    tt = sum(np.shape(ds.t.values))
    ds.close()
    
    UVbar = np.sqrt(np.power(Uav,2) + np.power(Vav,2))
    dsUVbar = xr.DataArray(data = UVbar.astype('float32'), name = "UVbar", dims = ['t', 'j', 'i'])

    ds = ds.merge(dsUVbar).drop_vars(['U', 'V', 'W'])

    ds.to_netcdf(gr_int + month[mo] + "_2D.nc")
    print("New grid saved to NetCDF")

    # Depth-averaged velocity
    UVbarstats = np.zeros(shape = (tt, 4))

    for ti in range(tt):
        UVbarstats[ti, :] = GeneralStats(UVbar[ti, :, :])

    UVbarstats = pd.DataFrame(UVbarstats)
    UVbarstats.to_csv(sm_path + "UVbar_stats_" + month[mo] + ".csv", index = False)

    del UVbarstats, UVbar, dsUVbar, Uav, Vav, tt

Skipped!


# Load and save density scenario data

The numpys are saved so that they can be generated in one big go and called later.

In [26]:
%%script echo Skipped!

# Load macroalgae density data
D0 = np.genfromtxt(densitydata, delimiter = ',')[1:186,3]

for mo in range(0,6):
    ds = xr.open_dataset(gr_raw + month[mo] + ".nc").load()
    tt = sum(np.shape(ds.t.values))
    ii = sum(np.shape(ds.i.values))
    jj = sum(np.shape(ds.j.values))
    kk = sum(np.shape(ds.k.values))
    ds.close()
    del ds
    
    # Current Dx and Dy
    Dx = np.linspace(1, days[mo], days[mo])
    Dy = D0[cday[mo]:(cday[mo]+days[mo])]
    
    # New Dx and Dy
    Dx1 = np.linspace(1, days[mo], tt)
    Dy1 = np.interp(Dx1, Dx, Dy)
    
    # Extend D to correct dimensions
    Dy1 = np.repeat(Dy1[:, np.newaxis], jj, axis = 1)
    Dy1 = np.repeat(Dy1[:, :, np.newaxis], ii, axis = 2)
    Dy1 = np.repeat(Dy1[:, np.newaxis, :, :], kk, axis = 1)
    
    # Save one empty scenario
    D2 = np.zeros([tt, kk, jj, ii])
    np.save(sc_path + "D_" + month[mo] + "_S0.npy", D2.astype('float32'))
    del D2
    
    # For each scenario
    for sc in range(1, 7):
        sc_c = np.load(sc_path + scenario[sc] + ".npy")
        sc_c = np.repeat(sc_c[None, :, :], tt, axis = 0)

        sc_c_1 = np.repeat(sc_c[:, None, :, :], 7, axis = 1)  # canopy spans depth layers 4-10 inclusive
        sc_c = np.concatenate((np.zeros([tt, 4, jj, ii]), sc_c_1, np.zeros([tt, (kk-4-7), jj, ii])), axis = 1)
        
        D3 = Dy1 * sc_c
        np.save(sc_path + "D_" + month[mo] + "_" + scenario[sc] + ".npy", D3.astype('float32'))
        
        print("Density data shaped and saved for " + month[mo] + ", " + scenario[sc])
        del sc_c, D3, sc_c_1

Skipped!


# Calculate attenuation

Attenuation changes with time and kelp density (D), depth-averaged velocity (UV-bar), depth (within or beneath canopy) and total water depth (botz). 

Technically this is a little bit of a stretch, as Plew's attenuation equation is only intended to come up with a total depth-averaged drag coefficient (Ct) which is a combination of u_c and u_b, and it's not meant to be literally applied to the within-canopy and beneath-canopy velocities seperately. However, it does give relative attenuation coefficients that are very reasonable when compared to actual data, so a case can be made that it's a good enough approximation. I think I can swing it alright if I focus on the big picture, or if it ends up being too iffy I can just run the model in 2D model and depth-average everything - I'll lose a little nuance but that's OK.

In [27]:
%%script echo Skipped!
aa = pow(0.2,2)
Cb = 0.0025

dscoords = xr.open_dataset(coordfile, decode_times = False).load()
botz = dscoords.botz.values
dscoords.close()
del dscoords

for mo in range(0,6):
    print("Starting " + month[mo] + "...")

    ds3D = xr.open_dataset(gr_raw + month[mo] + ".nc", decode_times = False).load()
    kk = sum(np.shape(ds3D.k.values))
    ds3D.close()
    
    ds2D = xr.open_dataset(gr_int + month[mo] + "_2D.nc", decode_times = False).load()
    UVbar = ds2D.UVbar.values
    tt = sum(np.shape(ds2D.t.values))
    ii = sum(np.shape(ds2D.i.values))
    jj = sum(np.shape(ds2D.j.values))
    ds2D.close()

    for sc in range(0,7):
        print("Starting scenario " + scenario[sc] + "...")
        
        uc_ub = np.ones(shape = (tt, kk, jj, ii))
        Ct = np.zeros(shape = (tt, jj, ii))
        
        D_c = np.load(sc_path + "D_" + month[mo] + "_" + scenario[sc] + ".npy")
        
        count = 0
        milcount = 0
        
        for xi in range(ii):  
            for yi in range(jj):    
                wz = abs(botz[yi, xi])
                H = 6.4/wz
                for ti in range(tt):  
                    D = D_c[ti, 7, yi, xi] # canopy density
                    UV0 = UVbar[ti, yi, xi]
                    Kd = 0.5 * wz * D * 0.0045 * pow(UV0,(1.13-2)) # D already includes the extra 0.5
                    under = Kd*H*(1-H)*(Cb*H+aa)-(aa*Cb*H)
                    if under < 0:
                        uc = 1
                        ub = 1
                    else:
                        top = -aa-Cb*pow(H,2)+(1-H)*sqrt(under)
                        bot = Kd*H*pow((1-H),3)-aa-Cb*pow(H,3)
                        uc = top/bot
                        ub = (1-uc*H)/(1-H)
                    Ct[ti, yi, xi] = Kd*H*pow(uc,2) + Cb*pow(ub,2)
                    
                    for zi in range(kk): # Attenuation is depth-averaged but, seperated between canopy and under-canopy
                        if zi in canopy_range:
                            uc_ub[ti, zi, yi, xi] = uc
                        else:
                            uc_ub[ti, zi, yi, xi] = ub
                        
                        count += 1
                        if count == 10e6:
                            milcount += 10
                            print(milcount, " of ", round(tt*ii*jj*kk/1e6,2), " million cells processed")
                            count = 0
                    
        print("Attenuation for " + scenario[sc] + " calculated")
        
        ds_uc_ub = xr.DataArray(data = uc_ub.astype('float32'), name = "uc_ub", dims = ['t', 'k', 'j', 'i'])
        ds_uc_ub = ds3D.merge(ds_uc_ub)
        ds_uc_ub.to_netcdf(gr_int + month[mo] + "_" + scenario[sc] + "_3D.nc")

        ds_Ct = xr.DataArray(data = Ct.astype('float32'), name = "Ctbar", dims = ['t', 'j', 'i'])
        ds_Ct = ds2D.merge(ds_Ct)
        ds_Ct.to_netcdf(gr_int + month[mo] + "_" + scenario[sc] + "_2D.nc")
        
        print("Scenario " + scenario[sc] + " datasets created and saved")
        del ds_Ct, ds_uc_ub
        
        attenUV = uc_ub
        attenUV[attenUV == 1] = None
        Ct[Ct == 0.0025] = None
        
        # Attenuation and drag coefficient for each time mean (all lon+lat), sd, min, max
        uc_stats = np.zeros(shape = (tt, 4))
        ub_stats = np.zeros(shape = (tt, 4))
        Ct_stats = np.zeros(shape = (tt, 4))
        
        for ti in range(tt):
            uc_stats[ti, :] = GeneralStats(attenUV[ti, canopy_range, :, :])
            ub_stats[ti, :] = GeneralStats(attenUV[ti, under_range, :, :])
            Ct_stats[ti, :] = GeneralStats(Ct[ti, :, :])

        uc_stats = pd.DataFrame(uc_stats)
        uc_stats.to_csv(sm_path + "uc_stats_" + month[mo] + "_" + scenario[sc] + ".csv", index = False)
        
        ub_stats = pd.DataFrame(ub_stats)
        ub_stats.to_csv(sm_path + "ub_stats_" + month[mo] + "_" + scenario[sc] + ".csv", index = False)
        
        Ct_stats = pd.DataFrame(Ct_stats)
        Ct_stats.to_csv(sm_path + "Ct_stats_" + month[mo] + "_" + scenario[sc] + ".csv", index = False)

        del Ct, Ct_stats, uc_stats, ub_stats, attenUV, uc_ub
        print("Scenario " + scenario[sc] + " stats finished")
        
del ds3D, ds2D

Skipped!


# Convert depth-averaged attenuation to component attenuation



Need to calculate the attenuation component $u_c'$, so that the attenuated velocity components $U_c=U*u_c'$ and $V_c=V*u_c'$ add up to the depth-averaged attenuation: $u_c'=\frac{\overline{UV_c}}{\sqrt{U^2+V^2}}$

In [28]:
%%script echo Skipped!
# atten = depth-averaged attenuation (either u_c or u_b)
# comp = attenuation component (for U and V)
# att_c = array of comp

import warnings
warnings.filterwarnings("error")

for mo in range(1,6):
    for sc in range(0,7):
        print("Starting " + month[mo] + ", " + scenario[sc] + "...")
        # Load component velocities
        ds3D = xr.open_dataset(gr_int + month[mo] + "_" + scenario[sc] + "_3D.nc", decode_times = False).load()
        U = ds3D.U.values
        V = ds3D.V.values
        uc_ub = ds3D.uc_ub.values.astype('float32')
        tt = sum(np.shape(ds3D.t.values))
        ii = sum(np.shape(ds3D.i.values))
        jj = sum(np.shape(ds3D.j.values))
        kk = sum(np.shape(ds3D.k.values))
        ds3D.close()
        
        # Load depth-averaged velocity and attenuation (UV bar is identical for every scenario)
        ds2D = xr.open_dataset(gr_int + month[mo] + "_" + scenario[sc] + "_2D.nc", decode_times = False).load()
        UVbar = ds2D.UVbar.values.astype('float32')
        Uav = ds2D.Uav.values.astype('float32')
        Vav = ds2D.Vav.values.astype('float32')
        ds2D = ds2D.drop_vars(['Uav', 'Vav'])
        ds2D.close()
        
        UVbar_z = np.repeat(UVbar[:, np.newaxis, :, :], repeats = kk, axis = 1)

        UVbar_atten = np.zeros(shape = (tt, kk, jj, ii)).astype('float32')
        att_c = np.ones(shape = (tt, kk, jj, ii)).astype('float32')
        V_atten = np.zeros(shape = (tt, kk, jj, ii)).astype('float32')
        U_atten = np.zeros(shape = (tt, kk, jj, ii)).astype('float32')
        
        count = 0
        milcount = 0
        
        for xi in range(ii):  
            for yi in range(jj):    
                for ti in range(tt):        
                    for zi in range(kk):
                        if zi in canopy_range:
                            atten = uc_ub[ti, zi, yi, xi]
                        else:
                            atten = uc_ub[ti, zi, yi, xi]
                        
                        # Apply depth-averaged attenuation to depth-averaged velocity
                        UVbar_atten[ti, zi, yi, xi] = UVbar_z[ti, zi, yi, xi] * atten
                        
                        if atten == 1:
                            comp = 1
                        else: # Calculate component velocities
                            try:
                                comp = UVbar_atten[ti, zi, yi, xi]/(sqrt(pow(Uav[ti, yi, xi],2) + pow(Vav[ti, yi, xi],2)))
                            except RuntimeWarning:
                                if Uav[ti, yi, xi] != 0 and Vav[ti, yi, xi] != 0:
                                    print("Divide by zero error at t=%f, k=%f, j=%f, i=%f, with UVbar=%f, U=%f, V=%f" 
                                          %(round(ti), round(zi), round(yi), round(xi), UVbar_atten[ti, zi, yi, xi], U[ti, zi, yi, xi], V[ti, zi, yi, xi]))
                                    break
                                else:
                                    comp = 1
                        
                        V_atten[ti, zi, yi, xi] = V[ti, zi, yi, xi] * comp
                        U_atten[ti, zi, yi, xi] = U[ti, zi, yi, xi] * comp
                        att_c[ti, zi, yi, xi] = comp
                        
                        count += 1
                        if count == 10e6:
                            milcount += 10
                            print(milcount, " of ", round(tt*ii*jj*kk/1e6,2), " million cells processed")
                            count = 0
        
        dsatten = xr.Dataset(
            data_vars = {'V_atten': (['t','k','j','i'], V_atten.astype('float32')),
                         'U_atten': (['t','k','j','i'], U_atten.astype('float32')),
                         'att_c': (['t','k','j','i'], att_c.astype('float32'))
                        })
        dsfin = ds2D.merge(dsatten)
        dsfin = ds3D.merge(dsfin)
        dsfin.to_netcdf(gr_fin + month[mo] + "_" + scenario[sc] + ".nc")

dsfin

Skipped!


## Fix times in the final grids

Not sure if this is even necessary.

In [29]:
#%%script echo Skipped!
times_may = np.linspace(start = 0, num = 4464, 
                        stop = dati.timedelta(days = 30, hours = 23, minutes = 50).total_seconds())
times_jun = np.linspace(start = times_may[-1] + dati.timedelta(minutes = 10).total_seconds(), num = 4320, 
                        stop = times_may[-1] + dati.timedelta(days = 30).total_seconds())
times_jul = np.linspace(start = times_jun[-1] + dati.timedelta(minutes = 10).total_seconds(), num = 4464, 
                        stop = times_jun[-1] + dati.timedelta(days = 31).total_seconds())
times_aug = np.linspace(start = times_jul[-1] + dati.timedelta(minutes = 10).total_seconds(), num = 4464, 
                        stop = times_jul[-1] + dati.timedelta(days = 31).total_seconds())
times_sep = np.linspace(start = times_aug[-1] + dati.timedelta(minutes = 10).total_seconds(), num = 4320, 
                        stop = times_aug[-1] + dati.timedelta(days = 30).total_seconds())
times_oct = np.linspace(start = times_sep[-1] + dati.timedelta(minutes = 10).total_seconds(), num = 4464, 
                        stop = times_sep[-1] + dati.timedelta(days = 31).total_seconds())

for sc in range(7):
    print("Starting scenario ", scenario[sc], "...")
    
    ds = xr.open_dataset(gr_fin + month[0] + "_" + scenario[sc] + ".nc", decode_times = False).load()
    ds.close()
    ds.times_secs.values = times_may
    ds.to_netcdf(gr_fin + month[0] + "_" + scenario[sc] + "_1.nc")
    print("May saved sucessfully")

    ds = xr.open_dataset(gr_fin + month[1] + "_" + scenario[sc] + ".nc", decode_times = False).load()
    ds.close()
    ds.times_secs.values = times_jun
    ds.to_netcdf(gr_fin + month[1] + "_" + scenario[sc] + "_1.nc")
    print("June saved sucessfully")

    ds = xr.open_dataset(gr_fin + month[2] + "_" + scenario[sc] + ".nc", decode_times = False).load()
    ds.close()
    ds.times_secs.values = times_jul
    ds.to_netcdf(gr_fin + month[2] + "_" + scenario[sc] + "_1.nc")
    print("July saved sucessfully")

    ds = xr.open_dataset(gr_fin + month[3] + "_" + scenario[sc] + ".nc", decode_times = False).load()
    ds.close()
    ds.times_secs.values = times_aug
    ds.to_netcdf(gr_fin + month[3] + "_" + scenario[sc] + "_1.nc")
    print("August saved sucessfully")

    ds = xr.open_dataset(gr_fin + month[4] + "_" + scenario[sc] + ".nc", decode_times = False).load()
    ds.close()
    ds.times_secs.values = times_sep
    ds.to_netcdf(gr_fin + month[4] + "_" + scenario[sc] + "_1.nc")
    print("September saved sucessfully")
    
    ds = xr.open_dataset(gr_fin + month[5] + "_" + scenario[sc] + ".nc", decode_times = False).load()
    ds.close()
    ds.times_secs.values = times_oct
    ds.to_netcdf(gr_fin + month[5] + "_" + scenario[sc] + "_1.nc")
    print("October saved sucessfully")

del ds, times_may, times_jun, times_jul, times_aug, times_sep, times_oct

Starting scenario  S0 ...
May saved sucessfully
June saved sucessfully
July saved sucessfully
August saved sucessfully
September saved sucessfully
October saved sucessfully
Starting scenario  S1 ...
May saved sucessfully
June saved sucessfully
July saved sucessfully
August saved sucessfully
September saved sucessfully
October saved sucessfully
Starting scenario  S2 ...
May saved sucessfully
June saved sucessfully
July saved sucessfully
August saved sucessfully
September saved sucessfully
October saved sucessfully
Starting scenario  S3 ...
May saved sucessfully
June saved sucessfully
July saved sucessfully
August saved sucessfully
September saved sucessfully
October saved sucessfully
Starting scenario  S4 ...
May saved sucessfully
June saved sucessfully
July saved sucessfully
August saved sucessfully
September saved sucessfully
October saved sucessfully
Starting scenario  S5 ...
May saved sucessfully
June saved sucessfully
July saved sucessfully
August saved sucessfully
September saved 